<a href="https://colab.research.google.com/github/MussieBirhane/Structural-Element-Identification/blob/main/CNN_Task5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [20]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import datasets, transforms
from tqdm.notebook import tqdm, trange

In [21]:
## Deep convolutional neural network
# Define the model

class CNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=5, padding=2)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=5, padding=2)
        #self.fc1 = nn.Linear(7 * 7 * 64, 256)
        self.fc1 = nn.Linear(64 * 56 * 56, 256)
        self.fc2 = nn.Linear(256, 3)

    def forward(self, x):
        # conv layer 1
        x = self.conv1(x)
        x = F.relu(x)
        x = F.max_pool2d(x, kernel_size=2)

        # conv layer 2
        x = self.conv2(x)
        x = F.relu(x)
        x = F.max_pool2d(x, kernel_size=2)

        #print(x.shape)

        # fc layer 1
        #x = x.view(-1, 7 * 7 * 64)
        x = x.view(-1, 64 * 56 * 56)
        x = self.fc1(x)
        x = F.relu(x)

        # fc layer 2
        x = self.fc2(x)
        return x

In [22]:
## Load the data
# training data
task5_train = np.load('/content/drive/MyDrive/task5_X_train.npy', allow_pickle=False)
task5_train_label = np.load('/content/drive/MyDrive/task5_y_train.npy', allow_pickle=False)

# testing data
task5_test = np.load('/content/drive/MyDrive/task5_X_test.npy', allow_pickle=False)
task5_test_label = np.load('/content/drive/MyDrive/task5_y_test.npy', allow_pickle=False)

# Transform the data ToTensor()
task5_train = torch.Tensor(task5_train)
task5_train_label = torch.Tensor(task5_train_label)
task5_test = torch.Tensor(task5_test)
task5_test_label = torch.Tensor(task5_test_label)

# Permute the tensor to feed into CNN model
task5_train = task5_train.permute(0, 3, 1, 2)
print('Permuted shape:', task5_train.shape)
task5_test = task5_test.permute(0, 3, 1, 2)

train_loader = torch.utils.data.DataLoader(task5_train, batch_size=100, shuffle=True)
train_label_loader = torch.utils.data.DataLoader(task5_train_label, batch_size=100, shuffle=True)
test_loader = torch.utils.data.DataLoader(task5_test, batch_size=64, shuffle=False)
test_label_loader = torch.utils.data.DataLoader(task5_test_label, batch_size=64, shuffle=False)


Permuted shape: torch.Size([1226, 3, 224, 224])


In [23]:
## Training
# Instantiate model
model = CNN()                                                       # CNN Model

# Loss and Optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)          # Adam optimizer

num_epochs = 5

# Iterate through train set minibatchs
for epoch in trange(num_epochs):
    for images, labels in tqdm(zip(train_loader, train_label_loader)):
        # Zero out the gradients
        optimizer.zero_grad()
        #print("images", images.shape)
        #print("labels", labels.shape)

        # Forward pass
        x = images
        y = model(x)
        #print(y.shape)
        loss = criterion(y, labels)
        # Backward pass
        loss.backward()
        optimizer.step()

        #print (f'Epoch [{epoch+1}/{num_epochs}], Step [{epoch+1}/{len(train_loader)}], Loss: {loss.item():.4f}')


  0%|          | 0/5 [00:00<?, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

In [24]:

## Testing
correct = 0
total = len(task5_test)

with torch.no_grad():
    # Iterate through test set minibatchs
    for images, labels in tqdm(zip(test_loader, test_label_loader)):
        # Forward pass
        x = images
        y = model(x)
        #print("y of each iteration", y)
        p, predictions = torch.max(y, dim=1)
        l, lbl = torch.max(labels, dim=1)
        #print("predictions of each iteration:", predictions)
        #print("labels of each iteration:", labels)
        correct += torch.sum((predictions == lbl).float())

print('Test accuracy: {}'.format(correct/total))


0it [00:00, ?it/s]

Test accuracy: 0.45890411734580994
